In [10]:
import matplotlib.pyplot as plt
import ee
import joblib
import geemap
from matplotlib.colors import ListedColormap
import geemap, geopandas as gpd

In [2]:
# Authenticate and initialize the Earth Engine API
ee.Authenticate()
ee.Initialize(project='svevazanettieo1') #così apro il progetto esistente in GEE, non ne sto creando uno nuovo

In [ ]:
# Load model 
clf = joblib.load("random_forest_model_train+val.pkl")

In [3]:
sentinel2018 = ee.Image('projects/svevazanettieo1/assets/Dubai/sentinel2018')
sentinel2020 = ee.Image('projects/svevazanettieo1/assets/Dubai/sentinel2020')
sentinel2022 = ee.Image('projects/svevazanettieo1/assets/Dubai/sentinel2022')
sentinel2024 = ee.Image('projects/svevazanettieo1/assets/Dubai/sentinel2024')

AOI = ee.FeatureCollection('projects/svevazanettieo1/assets/Dubai/Area_of_interest_Dubai')

In [19]:
# Read with GeoPandas
gdf2018 = gpd.read_file("samples2018.geojson")
gdf2020 = gpd.read_file("samples2020.geojson")
gdf2022 = gpd.read_file("samples2022.geojson")
gdf2024 = gpd.read_file("samples2024.geojson")

In [ ]:
print('Shape 2018', gdf2018.shape)
print('Shape 2020', gdf2020.shape)
print('Shape 2022', gdf2022.shape)
print('Shape 2024', gdf2024.shape)

Shape 2018 (870468, 11)
Shape 2020 (870468, 11)
Shape 2022 (870468, 11)
Shape 2024 (870468, 11)
  id      B11      B12      B2      B3       B4      B8      ndbi      ndvi  \
0  0  0.60190  0.54590  0.1921  0.2813  0.38580  0.4638  0.131048  0.088780   
1  1  0.57955  0.52235  0.2078  0.2932  0.39425  0.4728  0.104250  0.091241   
2  2  0.57955  0.52235  0.2065  0.2911  0.39320  0.4658  0.109187  0.088507   
3  3  0.57955  0.52235  0.2065  0.2911  0.39320  0.4658  0.109187  0.088507   
4  4  0.57400  0.51545  0.1958  0.2792  0.38280  0.4630  0.108968  0.094393   

       ndwi                   geometry  
0 -0.242733  POINT (55.60477 25.22159)  
1 -0.228513  POINT (55.60486 25.22159)  
2 -0.232004  POINT (55.60495 25.22159)  
3 -0.232004  POINT (55.60504 25.22159)  
4 -0.245828  POINT (55.60513 25.22159)  


In [22]:
def preprocess(gdf):
    gdf.columns = [col.replace('B', 'properties.B') for col in gdf.columns]
    gdf.columns = [col.replace('n', 'properties.n') for col in gdf.columns]
    feature_columns = [col for col in gdf.columns if col.startswith('properties.B') or col.startswith('properties.n')]
    whole_image = gdf[feature_columns]
    return whole_image

In [23]:
image2018 = preprocess(gdf2018)
image2020 = preprocess(gdf2020)
image2022 = preprocess(gdf2022)
image2024 = preprocess(gdf2024)

In [ ]:
scaler = joblib.load("scaler.save")

image2018 = scaler.transform(image2018)
image2020 = scaler.transform(image2020)
image2022 = scaler.transform(image2022)
image2024 = scaler.transform(image2024)

In [ ]:
y_pred_2018 = clf.predict(image2018)
gdf2018['predicted_class'] = y_pred_2018

y_pred_2020 = clf.predict(image2020)
gdf2020['predicted_class'] = y_pred_2020

y_pred_2022 = clf.predict(image2022)
gdf2022['predicted_class'] = y_pred_2022

y_pred_2024 = clf.predict(image2024)
gdf2024['predicted_class'] = y_pred_2024


In [ ]:
# definisco la lista di colori nell'ordine delle classi
colors = ['green', 'cyan', 'yellow', 'red']  

# creo la colormap discreta
cmap = ListedColormap(colors)

fig, axes = plt.subplots(2, 2, figsize=(10, 10))  # griglia 2x2
axes = axes.flatten()  # trasformo in lista per scorrere facilmente

title = ["2018", "2020", "2022", "2024"]
gdf=[gdf2018, gdf2020, gdf2022, gdf2024]

for i in range(len(title)):
    gdf[i].plot(
        column='predicted_class',
        cmap=cmap,
        legend=True,
        categorical=True,
        ax=axes[i]
    )
    axes[i].set_title(title[i])
    axes[i].set_axis_off()

plt.tight_layout()
plt.show()
